<a href="https://colab.research.google.com/github/pjseoane/AlgoTrading/blob/master/Packs_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pandas_datareader import data as pdr
#https://pandas-datareader.readthedocs.io/en/latest/remote_data.html

import collections

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime
#>>> datetime.today().strftime('%Y-%m-%d')
#'2021-01-26'

In [2]:
!pip install yfinance

In [3]:
import yfinance as yf
#https://pypi.org/project/yfinance/
yf.pdr_override() # <== that's all it takes :-)

In [4]:
class cNewPack():
  def __init__(self, pack_dictionary,start_date,end_date):
    
    self.pack_dictionary=pack_dictionary
    self.start_date=start_date
    self.end_date=end_date

    sortedPack=collections.OrderedDict(sorted(self.pack_dictionary.items())) 
    sortedTupleKeys=list(sortedPack.keys())
    sortedTupleValues=list(sortedPack.values())
    CCL=['GGAL.BA','GGAL']

    self.panel_stocks = pdr.get_data_yahoo(sortedTupleKeys, start=self.start_date, end=self.end_date)
    self.panel_CCL=pdr.get_data_yahoo(CCL, start=self.start_date, end=self.end_date)
    self.panel_CCL['FX']=self.panel_CCL.Close['GGAL.BA']*10/self.panel_CCL.Close['GGAL']

        
    self.panel_stocks_wgt=self.panel_stocks.Close * sortedTupleValues
    self.panel_stocks_wgt['Pack_USD']=self.panel_stocks_wgt.sum(axis=1)

    self.panel_stocks_wgt['CCL']=self.panel_CCL.FX

    self.panel_stocks_wgt['Pack_ARS']=self.panel_stocks_wgt.Pack_USD*self.panel_stocks_wgt.CCL

    #para llenar los gaps
    self.panel_normalized=self.panel_stocks_wgt.ffill().bfill()

  def getPackEvolution(self):
    return self.panel_normalized

  def getPanelCCL(self):
    return self.panel_CCL.FX

  def getPanelStocks(self):
    return self.panel_stocks

  def getPackPlot(self):
    #https://www.geeksforgeeks.org/how-to-add-a-y-axis-label-to-the-secondary-y-axis-in-matplotlib/
    
    y1=self.panel_normalized['Pack_ARS']
    y2=self.panel_normalized['CCL']
    fig, ax=plt.subplots(figsize = (10, 5))
    plt.title('Packs vs CCL')

    ax.set_xlabel('Fechas', color = 'r')
    ax2=ax.twinx()
    ax.set_ylabel('Pack ARS', color = 'g')
    ax2.set_ylabel('CCL', color = 'b')
    
    ax.plot(y1,color='g')
    ax2.plot(y2,color='b')

    plt.tight_layout()
    plt.show()

    return

  def writePanel_to_CSV(self):
    self.panel_normalized.to_csv('pack table', index=None)
    return

  def getCorrelationMatrix(self):
    return self.panel_stocks.Close.corr()

  def getStocksRecommendation(self):
    #arma un dict con toda la info de las empresas
    dict_info={}
    for key,values in self.pack_dictionary.items():
      try:
        dict_info[key]=yf.Ticker(key).info
      except:
        exit

    dict_calificaciones={}
    for key,values in dict_info.items():
      try:
        dict_calificaciones[key]=dict_info[key]['recommendationMean']
      
      except:
        exit
    return dict_calificaciones   

  


In [5]:
Pack_5G_dict={'AAPL': 8/10,'QCOM': 8/11, 'ADI': 2/3, 'AMD': 1/0.5,'SPY':0}
start_date = '2021-05-07'
#end_date = '2021-08-10'
end_date=datetime.today().strftime('%Y-%m-%d')
Pack_5G=cNewPack(Pack_5G_dict,start_date,end_date)

[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  2 of 2 completed


In [6]:
sr=Pack_5G.getStocksRecommendation()

In [11]:
sr

{'AAPL': 2, 'ADI': 1.7, 'AMD': 2.3, 'QCOM': 2}

In [8]:
sum(Pack_5G_dict.values())

4.193939393939393

In [10]:
recomendWgt=0
tot=sum(Pack_5G_dict.values())
for key,values in Pack_5G_dict.items():
      try:
        recomendWgt+=Pack_5G_dict[key]* sr[key]
      except:
        exit
recomendWgt/sum(Pack_5G_dict.values())

2.095375722543353

In [ ]:
sortedR=collections.OrderedDict(sorted(sr.items())) 
sortedRKeys=list(sortedR.keys())
sortedRValues=list(sortedR.values())
sortedRValues

In [ ]:
sortedPack=collections.OrderedDict(sorted(Pack_5G_dict.items())) 
sortedTupleKeys=list(sortedPack.keys())
sortedTupleValues=list(sortedPack.values())
sortedTupleValues

In [ ]:
sortedTupleCalifications

In [ ]:

#arma un dict con toda la info de las empresas
dict_info={}
for key,values in Pack_5G_dict.items():
  try:
    #Dict_calificaciones[key]=yf.Ticker(key).info['recommendationMean']
    dict_info[key]=yf.Ticker(key).info
  except:
    #print("error")
    exit 

In [ ]:
dict_calificaciones={}
for key,values in dict_info.items():
  try:
    dict_calificaciones[key]=dict_info[key]['recommendationMean']
    #print(dict_info[key]['recommendationMean'])
  except:
    exit
dict_calificaciones

In [ ]:
dict_info['AMD']['recommendationMean']

In [ ]:
Pack_5G.getPackEvolution().describe()

In [ ]:
Pack_5G.getPackEvolution()

In [ ]:
Pack_5G.getPanelCCL()

In [ ]:
Pack_5G.getPanelStocks()

In [ ]:
Pack_5G.getPackPlot()

In [ ]:
Pack_5G.getCorrelationMatrix()

In [ ]:
ticker=yf.Ticker('QCOM')
ticker.history(period="1y")

In [ ]:
Pack_IAv1={'NVDA': 12/24,'FB': 2/8, 'GOOGL': 2/58, 'AMZN': 3/144}
start_date = '2021-05-14'
datetime.today().strftime('%Y-%m-%d')
Pack_IAv1=cNewPack(Pack_IAv1,start_date,end_date)

In [ ]:
Pack_IAv1.getPackEvolution()

In [ ]:
Pack_IAv1.getPackPlot()

In [ ]:
Pack_IAv1.getCorrelationMatrix()